# Model of product recomendation

Goal of given model - to improve prediction by adding SEX column

Content

1. Data Upload
2. Add activity_column
3. Add Gender Column
4. Replace NaN (gender by customer purchaising history)

5. Process Datetime column
6. Label Encode (customer #)

7. Preparation for Baseline accuracy prediction
 - 7.1 One-Hot Encode (without sex column)

8. Preparation for Target accuracy prediction
 - 8.1 One-Hot Encode (with sex column)


9. GridSearchCV_for_baseline 

10. GridSearchCV_for_target(with sex column)

11. Final accuracy prediction

12. Conclusion

## Libraries 

In [82]:
import pandas as pd 
import numpy as np

import statistics

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import accuracy_score


from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier


# 1. Data Uploading

In [83]:
train = pd.read_csv("purchases_train.csv").sort_values(by='datetime',ascending=True)
test = pd.read_csv("purchases_test.csv").sort_values(by='datetime',ascending=True)
cust = pd.read_csv("customers.csv",na_values='nan')

y_train=train['product_id'].copy()
y_test=test['product_id'].copy()

#train=train.drop(['product_id'],axis=1)
#test=test.drop(['product_id'],axis=1)

print('Train',train.shape)
print('Train',train.columns)
print('Test',test.shape)
print('Customers',cust.shape)


Train (351686, 3)
Train Index(['customer_id', 'product_id', 'datetime'], dtype='object')
Test (21935, 3)
Customers (123565, 2)


In [84]:
all_data=pd.concat([train,test])

# 2.Add activity Column

In [85]:
#Activity- frequency of customer in train dataset
activity=pd.DataFrame(train['customer_id'].value_counts()).reset_index()
activity.columns=['customer_id','activity']

In [86]:
all_data1=pd.merge(all_data, activity, left_on='customer_id', right_on='customer_id')

In [87]:
all_data=all_data1.copy()

# 3. Add Gender Column

In [88]:
all_data=all_data.merge(cust, on='customer_id', how='left')
all_data=all_data.sort_values(by='datetime',ascending=True)
print('All_data',all_data.shape)
print('All_data',all_data.columns)

All_data (373621, 5)
All_data Index(['customer_id', 'product_id', 'datetime', 'activity', 'sex'], dtype='object')


In [89]:
#Unique_customer_id
print(train['customer_id'].max())
print(train['customer_id'].nunique())

21696337
107491


In [90]:
#No_Sex_amount_rows
all_data['noSex-1/0']=0
all_data['noSex-1/0'] = all_data['sex'].apply(lambda x: 1 if pd.isnull(x) else 0)

all_data['noSex-1/0'].value_counts()

0    225145
1    148476
Name: noSex-1/0, dtype: int64

In [91]:
#Null_values
all_data.isnull().sum()

customer_id         0
product_id          0
datetime            0
activity            0
sex            148476
noSex-1/0           0
dtype: int64

# 4. Replace NaN_SEX

## 4.1 Replace by customer purchaising history

In [92]:
#Product_list with_NaN-unique product ID

prod_list_sex_NAN=all_data.loc[all_data['sex'].isnull()]
print('Uniq_Nan_products',prod_list_sex_NAN['product_id'].nunique())
print('Total NaN rows',prod_list_sex_NAN['product_id'].shape)

uniq_na_sex_prod=pd.DataFrame(prod_list_sex_NAN['product_id'].unique())
uniq_na_sex_prod.columns=['product_id']
uniq_na_sex_prod['NA_Sex']='yes'

Uniq_Nan_products 1653
Total NaN rows (148476,)


In [93]:
#Male-Female distribution
all_data['sex'].value_counts()

Female    169747
Male       55398
Name: sex, dtype: int64

In [94]:
#Add column - How many times this product was bought
freq=pd.DataFrame(all_data['product_id'].value_counts()).reset_index()
freq.columns=['product_id','product_count_total']

all_data=all_data.merge(freq, on='product_id', how='left')

all_data.isnull().sum()


customer_id                 0
product_id                  0
datetime                    0
activity                    0
sex                    148476
noSex-1/0                   0
product_count_total         0
dtype: int64

In [95]:
#Create table - product_id - predicted SEX (by purchaising history)
gender=all_data.groupby(['sex','product_id'],as_index=False).count()
gender=gender.drop(['datetime','noSex-1/0','product_count_total','activity'],axis=1)

gender=gender.merge(freq, on='product_id', how='right')
gender.columns=['sex','product_id','prod_times_bought_by_sex','Prod_freq']
gender['%_this_prod_by_sex_bought']=round(gender['prod_times_bought_by_sex']*100/gender['Prod_freq'],2)
gender=gender.dropna()

gender=gender.merge(uniq_na_sex_prod, on='product_id', how='left')
gender=gender.sort_values(by=['product_id','sex'])
gender=gender.drop(['NA_Sex'],axis=1)
gender['sex_pred']=0
a=[]
for i in list(gender['product_id']):
        #Найти макс значение продажи продукта по полу и знач пола
        prod_sex_2=gender.loc[gender['product_id']==i]
            #m=gender.loc[prod_sex_2['%_this_prod_by_sex_bought'].max]

        ind_max=prod_sex_2['%_this_prod_by_sex_bought'].idxmax(axis=1, skipna=True)
        max_sex=gender['sex'][ind_max]

        a.append(max_sex)

gender['sex_pred']=pd.DataFrame(a)
gender.drop_duplicates(subset ="product_id", inplace = True)
gender=gender.drop(['sex'],axis=1)
gender

,product_id,prod_times_bought_by_sex,Prod_freq,%_this_prod_by_sex_bought,sex_pred
0,15,294.0,450,65.33,Female
2,22,43.0,68,63.24,Female
4,25,11.0,22,50.00,Female
6,27,289.0,932,31.01,Female
8,49,2.0,24,8.33,Female
...,...,...,...,...,...
3248,13762,5.0,33,15.15,Female
3250,13768,5.0,23,21.74,Female
3252,13802,1.0,1,100.00,Female
3296,13815,3.0,4,75.00,Female


In [96]:
#Vlookup all_data and Predicted-Sex
#120 prod_id still no SEX  
all_data_sex_pr=all_data.merge(gender, on='product_id', how='left')
all_data_sex_pr=all_data_sex_pr.drop(['prod_times_bought_by_sex','Prod_freq','%_this_prod_by_sex_bought'],axis=1)

all_data_sex_pr['noSex_after_pred-1/0']=0
all_data_sex_pr['noSex_after_pred-1/0'] = all_data_sex_pr['noSex_after_pred-1/0'].apply(lambda x: 1 if pd.isnull(x) else 0)

print(all_data_sex_pr.shape)
all_data_sex_pr.isnull().sum()

(373621, 9)


customer_id                  0
product_id                   0
datetime                     0
activity                     0
sex                     148476
noSex-1/0                    0
product_count_total          0
sex_pred                   120
noSex_after_pred-1/0         0
dtype: int64

In [97]:
#ALl Sex to one columns - given+predicted
all_data_sex_pr['final_sex']=all_data_sex_pr['sex']
all_data_sex_pr['final_sex'].fillna(all_data_sex_pr['sex_pred'],inplace=True)
print(all_data_sex_pr.shape)
all_data_sex_pr.isnull().sum()

(373621, 10)


customer_id                  0
product_id                   0
datetime                     0
activity                     0
sex                     148476
noSex-1/0                    0
product_count_total          0
sex_pred                   120
noSex_after_pred-1/0         0
final_sex                  120
dtype: int64

In [98]:
#REplace rest 120 NAN sex to 'Female'
all_data_sex_pr['final_sex']=all_data_sex_pr['final_sex'].fillna('Female')
print(all_data_sex_pr.shape)
all_data_sex_pr.isnull().sum()

(373621, 10)


customer_id                  0
product_id                   0
datetime                     0
activity                     0
sex                     148476
noSex-1/0                    0
product_count_total          0
sex_pred                   120
noSex_after_pred-1/0         0
final_sex                    0
dtype: int64

In [99]:
all_data_sex_pr['final_sex'].value_counts()

Female    304262
Male       69359
Name: final_sex, dtype: int64

In [100]:
#Final sex - 1/0
all_data_sex_pr['final_sex'] = all_data_sex_pr['final_sex'].map(lambda x: 1 if x == 'Male' else 0)

In [101]:
#keep Final Sex column - separate for accuracy check
sex_df=pd.DataFrame(all_data_sex_pr['final_sex'])


# 5. Process Datetime

In [102]:
#Datetime
all_data['datetime'] = pd.to_datetime(all_data['datetime'],format='%Y-%m-%d %H:%M:%S')
all_data['year']=all_data['datetime'].dt.year
all_data['month']=all_data['datetime'].dt.month
all_data['day']=all_data['datetime'].dt.day

all_data['wday']=all_data['datetime'].dt.dayofweek 
#all_data['weekday_name']=all_data['datetime'].dt.weekday

all_data['Hour'] = all_data['datetime'].dt.hour 
#all_data['minute'] = all_data['datetime'].dt.minute

In [103]:
print('All_data',all_data.shape)
print('All_data',all_data.columns)

All_data (373621, 12)
All_data Index(['customer_id', 'product_id', 'datetime', 'activity', 'sex', 'noSex-1/0',
       'product_count_total', 'year', 'month', 'day', 'wday', 'Hour'],
      dtype='object')


In [104]:
all_data=all_data.sort_values(by='datetime')

In [105]:
all_data=all_data.drop(['datetime'],axis=1)

# 6. LabelEnc

In [106]:
train['customer_id'].max()

21696337

In [107]:
#LabelEncoder - customer_id
le = LabelEncoder()
all_data['customer_id']=pd.DataFrame(le.fit_transform(all_data['customer_id']),columns=['customer_id'])
all_data['customer_id'].max()

107490

In [108]:
#split-train/test_easier_dummy
train=all_data[:len(y_train)].copy()
test=all_data[len(y_train):].copy()

print('Train',train.shape)
print('Test',test.shape)

Train (351686, 11)
Test (21935, 11)


In [109]:
all_data.columns

Index(['customer_id', 'product_id', 'activity', 'sex', 'noSex-1/0',
       'product_count_total', 'year', 'month', 'day', 'wday', 'Hour'],
      dtype='object')

# 7.Preparation for Baseline accuracy prediction

## 7.1 One_hot

In [110]:
#One_hot for categorical
categorical=['customer_id', 'year', 'month', 'day', 'wday', 'Hour']
not_categorical=['activity']

enc = OneHotEncoder(drop='first')
enc=enc.fit(pd.concat([train[categorical],test[categorical]]))
X_cat_sparse=enc.transform(train[categorical])
X_test_cat_sparse=enc.transform(test[categorical])

from scipy.sparse import hstack
X_sparse=hstack((train[not_categorical], X_cat_sparse))
X_test_sparse=hstack((test[not_categorical], X_test_cat_sparse))

print("Training data: " + format(X_sparse.shape))
print("Test data: " + format(X_test_sparse.shape))
print("###########")


Training data: (351686, 107556)
Test data: (21935, 107556)
###########


In [111]:
X_train=X_sparse
X_test=X_test_sparse

In [112]:
print('Train',X_train.shape)
print('Test',X_test.shape)

print('Train',y_train.shape)
print('Test',y_test.shape)

Train (351686, 107556)
Test (21935, 107556)
Train (351686,)
Test (21935,)


# 8. Preparation for Target accuracy prediction

## 8.1 One-Hot Encode (df with sex column (predicted))

In [113]:
all_data_sex=pd.concat([all_data,sex_df],axis=1)
print(all_data_sex.shape)

train_sex=all_data_sex[:len(y_train)].copy()
test_sex=all_data_sex[len(y_train):].copy()

print('Train',train_sex.shape)
print('Test',test_sex.shape)
print('Columns',all_data_sex.columns)

(373621, 12)
Train (351686, 12)
Test (21935, 12)
Columns Index(['customer_id', 'product_id', 'activity', 'sex', 'noSex-1/0',
       'product_count_total', 'year', 'month', 'day', 'wday', 'Hour',
       'final_sex'],
      dtype='object')


In [114]:
#One_hot for categorical
categorical=['customer_id', 'year', 'month', 'day', 'wday', 'Hour','final_sex']
not_categorical=['activity']

enc = OneHotEncoder(drop='first')
enc=enc.fit(pd.concat([train_sex[categorical],test_sex[categorical]]))

X_cat_sparse_sex=enc.transform(train_sex[categorical])
X_test_cat_sparse_sex=enc.transform(test_sex[categorical])

from scipy.sparse import hstack
X_sparse_sex=hstack((train_sex[not_categorical], X_cat_sparse_sex))
X_test_sparse_sex=hstack((test_sex[not_categorical], X_test_cat_sparse_sex))

print("Training data: " + format(X_sparse_sex.shape))
print("Test data: " + format(X_test_sparse_sex.shape))
print("###########")


Training data: (351686, 107557)
Test data: (21935, 107557)
###########


In [115]:
X_train_sex=X_sparse_sex
X_test_sex=X_test_sparse_sex

# 9.GridSearchCV_for_baseline

In [116]:
#Split dataset for Gridsearch
X=X_train
y=y_train
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, train_size=0.2, test_size=0.8,
                                                      random_state=241)

cv=KFold(n_splits=5,shuffle=True,random_state=241)

In [117]:
#Using GridSearchCV we defined the best parameters for baseline dataset

depth = [7]
crit=['gini']
sam=[5]
leaf=[2]

parameters = {'max_depth':depth,
              'criterion':crit,
             'min_samples_split':sam,
             'min_samples_leaf':leaf,
             }
model = DecisionTreeClassifier()
grid = GridSearchCV(model, parameters,scoring='accuracy', cv=cv)
grid.fit(X_train_,y_train_)
score_my=(grid.best_score_)
print('The parameters combination that would give best accuracy is : ')
print(grid.best_params_)
print('The best accuracy achieved after parameter tuning via grid search is : ', score_my)

The parameters combination that would give best accuracy is : 
{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 5}
The best accuracy achieved after parameter tuning via grid search is :  0.06545627157120987


# 10.GridSearchCV_for_target(with sex column)

In [118]:
X=X_train_sex
y=y_train
X_train_sex_, X_test_sex_, y_train_sex_, y_test_sex_ = train_test_split(X, y, train_size=0.2, test_size=0.8,
                                                      random_state=241)

In [119]:
#Using GridSearchCV we defined the best parameters for target dataset
depth = [9]
crit=['gini']
sam=[4]
leaf=[1]

parameters = {'max_depth':depth,
              'criterion':crit,
             'min_samples_split':sam,
             'min_samples_leaf':leaf,
}
model = DecisionTreeClassifier()
grid = GridSearchCV(model, parameters,scoring='accuracy', cv=cv)
grid.fit(X_train_sex_,y_train_sex_)
score_my=(grid.best_score_)
print('The parameters combination that would give best accuracy is : ')
print(grid.best_params_)
print('The best accuracy achieved after parameter tuning via grid search is : ', score_my)

The parameters combination that would give best accuracy is : 
{'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 4}
The best accuracy achieved after parameter tuning via grid search is :  0.07562161538188043


# 11. Final accuracy prediction

In [120]:
# Baseline
model_basic=DecisionTreeClassifier(max_depth=7,criterion='gini',min_samples_split=5,min_samples_leaf=2)

In [121]:
model_basic.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [123]:


model_basic_predictions = model_basic.predict(X_test)
acc = accuracy_score(model_basic_predictions, y_test)

pred_matrix=y_test==model_basic_predictions
print ('Baseline accuracy prediction with tune parameters',acc)
print('False/True predictions',pred_matrix.value_counts())

Baseline accuracy prediction with tune parameters 0.10558468201504445
False/True predictions False    19619
True      2316
Name: product_id, dtype: int64


In [124]:
#Target with sex
model_target=DecisionTreeClassifier(max_depth=9,criterion='gini',min_samples_split=4,min_samples_leaf=1)

In [125]:
model_target.fit(X_train_sex, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=9, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [126]:
model_target_predictions = model_target.predict(X_test_sex)
acc = accuracy_score(model_target_predictions, y_test)

pred_matrix=y_test==model_target_predictions
print ('Baseline accuracy prediction with tune parameters',acc)
print('False/True predictions',pred_matrix.value_counts())

Baseline accuracy prediction with tune parameters 0.10945976749487121
False/True predictions False    19534
True      2401
Name: product_id, dtype: int64


# 12.Conclusion

In [ ]:
Thus, by adding column 'Sex' to our dataset we managed to improve accuracy from 10.56% to 10.95% with knn=19.
It means that we improved amount of right predictions from 2316 to 2401.